In [16]:
import os
import numpy as np 
import tensorflow as tf
import seaborn as sns

from sklearn.metrics import confusion_matrix, classification_report


# Fix GDrive

In [ ]:
print("Installing G-Drive ... ")
os.system("pip install -U --no-cache-dir gdown --pre > /dev/null")

# Downoad Data

In [ ]:
os.system("gdown ---------------")
os.system('unzip -o "TAVI_STRESS_CLF_PARALLEL.zip" > /dev/null')

# Load Data

In [ ]:
DATA_DIR = "/content/Stress/"
train_X1 = np.load(DATA_DIR + "train_X1.npy")
train_X2 = np.load(DATA_DIR + "train_X2.npy")
train_X3 = np.load(DATA_DIR + "train_X3.npy")
train_Y = np.load(DATA_DIR + "train_Y.npy")
test_X1 = np.load(DATA_DIR + "test_X1.npy")
test_X2 = np.load(DATA_DIR + "test_X2.npy")
test_X3 = np.load(DATA_DIR + "test_X3.npy")
test_Y = np.load(DATA_DIR + "test_Y.npy")

# Config

In [7]:
IMG_SIZE = 224
BATCH_SIZE = 4
LEARNING_RATE = 3e-4
EPOCHS = 300
EPOCH_PATIENCE = 30


# Model Architecture

In [14]:
inputs = []
features = []

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)

for i in range(3):
    inp = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inp)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    inputs.append(inp)
    features.append(x)

x = tf.keras.layers.concatenate(features)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_15 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_16 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 tf.math.truediv_7 (TFOpLam  (None, 224, 224, 3)          0         ['input_14[0][0]']            
 bda)                                                                                         

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    [train_X1, train_X2, train_X3],
    train_Y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=([test_X1, test_X2, test_X3], test_Y),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=EPOCH_PATIENCE,
            restore_best_weights=True
        )
    ]
)

# Evaluation

In [ ]:
pred_y = model.predict([test_X1, test_X2, test_X3])
pred_y = np.round(pred_y).astype(int).reshape(-1)
test_y = test_Y.astype(int)

print(classification_report(test_y, pred_y))

cm = confusion_matrix(test_y, pred_y)
sns.heatmap(cm, annot=True, fmt="d")
